In [1]:
!curl  https://adventofcode.com/2021/day/9/input --silent --show-error --cookie session=(cat .session_token) --output inputs/09.txt

# Day 9: Smoke Basin

These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.

Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?

In [2]:
import numpy as np

In [34]:
with open("inputs/09.txt") as infile:
    heightmap = np.asarray([[int(x) for x in line] for line in infile.read().splitlines()], dtype=np.int8)

#     test="""2199943210
# 3987894921
# 9856789892
# 8767896789
# 9899965678"""

#     heightmap = np.asarray([[int(x) for x in line] for line in test.splitlines()], dtype=np.int8)

In [36]:
padded_heightmap = np.pad(heightmap, 1, constant_values=100)
windows = np.lib.stride_tricks.sliding_window_view(padded_heightmap, (3, 3))

# we can add this mask to any window to offset all values we're not interested in
# since the maximum value of the heightmap is 10, by setting the center and diagonal 
# values to a value >> 10 we make sure they are not considered when we find the minimum 
# number in the surrounding fields 
adjacency_mask =  [[100, 0, 100], [0, 100, 0], [100, 0, 100]]
lowest_adjacent = np.min(windows + adjacency_mask, axis=(2, 3))
centers = windows[:, :, 1, 1]
low_point_coords = np.where(centers < lowest_adjacent)
low_points = centers[low_point_coords]
risk_levels = low_points + 1
print(f"{np.sum(risk_levels)=}")


np.sum(risk_levels)=564


# Part Two

Next, you need to find the largest basins so you know what areas are most important to avoid.

A basin is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.

The size of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size 3:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
The top-right basin, size 9:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
The middle basin, size 14:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
The bottom-right basin, size 9:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.

What do you get if you multiply together the sizes of the three largest basins?

In [53]:
BASIN_MARGIN = 9
BASIN_FILL = 10

basin_map = np.pad(heightmap, 1, constant_values=BASIN_FILL)

def fill_basin_points(center_x, center_y):
    num_points_in_basin = 0
    if basin_map[center_x][center_y] in (BASIN_MARGIN, BASIN_FILL):
        return 0
    
    basin_map[center_x][center_y] = BASIN_FILL
    num_points_in_basin += 1
    num_points_in_basin += fill_basin_points(center_x - 1, center_y)
    num_points_in_basin += fill_basin_points(center_x + 1, center_y)
    num_points_in_basin += fill_basin_points(center_x, center_y - 1)
    num_points_in_basin += fill_basin_points(center_x, center_y + 1)

    return num_points_in_basin

basin_sizes = []
for x, y in zip(*low_point_coords):
    basin_size = fill_basin_points(x, y)
    basin_sizes.append(basin_size)

basin_sizes.sort(reverse=True)
three_largest_basins = basin_sizes[:3]
print(f"{np.product(three_largest_basins)=}")

np.product(three_largest_basins)=1038240
